In [1]:
from utils import *
import sys, os, random
# from compare_locations import compare_locations

datadir = "../data.upb/precis/01-precis-redmi-pixel4a/nowl/"
#datadir = "../data.upb/precis/02-precis-redmi-pixel4a/"



In [2]:
# parter - face level 
#pix_etaje = ["0-pixel-04-06-2021_19-34-41.json"]
#red_etaje = ["0-redmi-04-06-2021_19-33-47.json"]

# whole building - hip level 
pix_etaje = [
"0-pixel-04-03-2021_15-32-38.json.gz",
 "1-pixel-25-02-2021_21-16-46.json.gz",
 "2-pixel-25-02-2021_16-27-45.json.gz",
 "3-pixel-04-03-2021_16-18-50.json.gz",
 "4-pixel-04-03-2021_16-43-50.json.gz",
 "5-pixel-04-03-2021_17-11-32.json.gz",
 "6-pixel-17-03-2021_12-52-29.json.gz",
 "7-pixel-17-03-2021_13-14-36.json.gz"
]

red_etaje = [
"0-redmi-04-03-2021_15-31-12.json.gz"#,
# "1-redmi-25-02-2021_21-16-10.json",
# "2-redmi-25-02-2021_16-27-08.json",
# "3-redmi-04-03-2021_16-18-52.json",
# "4-redmi-04-03-2021_16-43-45.json",
# "5-redmi-04-03-2021_17-11-30.json",
# "6-redmi-17-03-2021_12-52-29.json",
# "7-redmi-17-03-2021_13-14-39.json"
]


retaje = []
for e in red_etaje: 
    json_file = os.path.join(datadir, e)
    etaj = load_dataset_json(json_file)
    retaje.append(etaj)

petaje = []
for e in pix_etaje: 
    json_file = os.path.join(datadir, e)
    etaj = load_dataset_json(json_file)
    petaje.append(etaj)
    

In [3]:
import numpy as np
from scipy.spatial.distance import braycurtis
from scipy.stats import ks_2samp, chisquare
import copy 

def merge_wifi_fingerprints(flist):
    fingerprint = copy.deepcopy(flist[0])
    for f2 in copy.deepcopy(flist[1:]):
        if not "wifi" in f2.keys():
            continue
        for mac in f2["wifi"].keys():
            if not mac in fingerprint["wifi"].keys():
                fingerprint["wifi"][mac] = f2["wifi"][mac]
                #fingerprint["wifi"][mac]['rssi'] = []
            else:
                fingerprint["wifi"][mac]['rssi'].extend(f2["wifi"][mac]['rssi'])
            fingerprint["wifi"][mac]['rssi'].sort()    
    return fingerprint


def compare_locations(c1, c2, simil_method = braycurtis,  selection = 'Average'):
    rssi1 = []
    rssi2 = []
    w = [] #weight for braycurtis
    
    # take the first fingerprint in the list 
    #wifi1 = c1['fingerprints'][0]['wifi']
    #wifi2 = c2['fingerprints'][0]['wifi']

    # take a random fingerprint in the list 
    #wifi1 = c1['fingerprints'][random.randint(0,3)]['wifi']
    #wifi2 = c2['fingerprints'][random.randint(0,3)]['wifi']

    # first & third - as if measuring twice at 180 degrees
    wifi1 = merge_wifi_fingerprints([c1['fingerprints'][0], c1['fingerprints'][2]])['wifi']
    wifi2 = merge_wifi_fingerprints([c2['fingerprints'][0], c2['fingerprints'][2]])['wifi']
    
    #merge all existing fingerprints for each collection
    #wifi1 = merge_wifi_fingerprints(c1['fingerprints'])['wifi']
    #wifi2 = merge_wifi_fingerprints(c2['fingerprints'])['wifi']
    
    # all power received by the query
    power1 = 0
    for ap in wifi1:
        power1 += np.sum(adjust_rssi(wifi1[ap]['rssi']))
        
    common_aps = list(set(wifi1.keys()) & set(wifi2.keys()))
    #print("common aps=", len(common_aps))
    
    # No APs in common -> similarity = 1
    if not common_aps:
        return 1

    # TODO: find the best metric
    # If not enough common APs -> similarity = 1
    if len(common_aps) * 10 < len(wifi1.keys()) or len(common_aps) < 3:
        return 1

    for ap in common_aps:
        
        if selection == 'Bestdir':
            for f1 in c1['fingerprints']: # all fingerprint dirs in the query 
                if not ap in f1['wifi'].keys():
                    continue
                m1 = np.average(adjust_rssi(f1['wifi'][ap]['rssi']))
                best = 100                
                for f2 in c2['fingerprints']: 
                    if not ap in f2['wifi'].keys():
                        continue
                    m2 = np.average(adjust_rssi(f2['wifi'][ap]['rssi']))
                    if(math.fabs(m2-m1)) < best:  # find one dir with closest RSSI
                        best = math.fabs(m2-m1)
                        bestf = m2
                rssi1.append(m1)
                rssi2.append(bestf)
                
        # Take only the first RSSI value                
        if selection == 'First':
            rssi1.append(wifi1[ap]['rssi'][0])
            rssi2.append(wifi2[ap]['rssi'][0])

        # Make an average of all RSSI values
        if selection == 'Average':
            rssi1.append(np.average(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.average(adjust_rssi(wifi2[ap]['rssi'])))
            w.append(min(len(wifi1[ap]['rssi']), len(wifi2[ap]['rssi'])))

        if selection == 'Median':
            rssi1.append(np.median(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.median(adjust_rssi(wifi2[ap]['rssi'])))

        if selection == 'Mean':
            rssi1.append(np.mean(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.mean(adjust_rssi(wifi2[ap]['rssi'])))

        if selection == 'Std':
            rssi1.append(np.std(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.std(adjust_rssi(wifi2[ap]['rssi'])))

        if selection == 'Max':
            rssi1.append(np.max(adjust_rssi(wifi1[ap]['rssi'])))
            rssi2.append(np.max(adjust_rssi(wifi2[ap]['rssi'])))
            
        if selection == 'KS':
            _, p = ks_2samp(wifi1[ap]['rssi'], wifi2[ap]['rssi'])
            rssi1.append(p)
            rssi2.append(1.0)

        if selection == 'Tempered':
            rss_1.append(np.average(rssi_v[index][key]) * random.uniform(0.8, 1.2))
            rss_2.append(np.average(rssi_v[r][key]) * random.uniform(0.8, 1.2))

    #print(f"1={rssi1}\n2={rssi2}\ncommon={common_aps}")
    if selection != 'KS':
        rssi1 = list(map(lambda x: x - np.mean(rssi1), rssi1))
        rssi2 = list(map(lambda x: x - np.mean(rssi2), rssi2))
    return simil_method(tuple(rssi1), tuple(rssi2))
#        return simil_method(tuple(rssi1), tuple(rssi2))*sum(rssi1)/power1
# weigh final result with used power/total power   

In [5]:
macs = []
for e in petaje:     
    macse = set([])
    for c in e:
        f = c['fingerprints']
        allf = merge_wifi_fingerprints(f)['wifi']
        macse = macse | set(allf.keys())
    macs.append(macse)    
    print("macs = ", len(macse))    
    
print("l = ", len(macs[0] & macs[2]))    
    
for me in macs:
    for ne in macs: 
        print("%2d " % len(me & ne), end='')
    print("")    

print("Difference:")
for me in macs:
    for ne in macs: 
        print("%2d " % len(me - ne), end='')
    print("")        

macs =  83
macs =  75
macs =  73
macs =  75
macs =  68
macs =  76
macs =  74
macs =  50
l =  27
83 49 27  3  0  0  0  0 
49 75 53 29  0  0  0  0 
27 53 73 48 19  0  0  0 
 3 29 48 75 44 24  0  0 
 0  0 19 44 68 48 24  0 
 0  0  0 24 48 76 49 25 
 0  0  0  0 24 49 74 50 
 0  0  0  0  0 25 50 50 
Difference:
 0 34 56 80 83 83 83 83 
26  0 22 46 75 75 75 75 
46 20  0 25 54 73 73 73 
72 46 27  0 31 51 75 75 
68 68 49 24  0 20 44 68 
76 76 76 52 28  0 27 51 
74 74 74 74 50 25  0 24 
50 50 50 50 50 25  0  0 
